In [1]:
import pandas as pd
import numpy as np
# import requests

In [2]:
df_jan = pd.read_parquet(path='https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet')
df_jan_og = df_jan.copy()
# df_jan = pd.read_parquet(r"C:\Users\Sarath.Lavu\Downloads\Learn\MLOps\Ass1\yellow_tripdata_2022-01.parquet")

In [3]:
df_jan_og.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


## Q1

In [4]:
print(f"Number of columns in Jan Yellow Taxi data = {df_jan_og.shape[1]}")

Number of columns in Jan Yellow Taxi data = 19


In [5]:
df_jan.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [6]:
df_jan.isna().sum()

VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          71503
trip_distance                0
RatecodeID               71503
store_and_fwd_flag       71503
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     71503
airport_fee              71503
dtype: int64

## Q2

In [7]:
def get_duration(df):
    df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
    return df

In [8]:
df_jan_og = get_duration(df_jan_og)

In [9]:
# df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60

In [10]:
print(f"Standard deviation of the trips duration in January = {np.std(df_jan_og['duration'])}")

Standard deviation of the trips duration in January = 46.445295712725304


## Q3

In [13]:
def remove_outliers(df):
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)].reset_index(drop=True)
    return df

In [14]:
prev_len = len(df_jan_og)

# df_jan = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)].reset_index(drop=True)
df_jan = remove_outliers(df_jan_og.copy())

without_outlier_len = len(df_jan)

In [15]:
without_outlier_len/prev_len

0.9827547930522406

In [16]:
print(f"Fraction of the records left after dropping the outliers = {without_outlier_len/prev_len}")

Fraction of the records left after dropping the outliers = 0.9827547930522406


## Q4

In [17]:
from sklearn.feature_extraction import DictVectorizer

In [18]:
df_jan[['PULocationID', 'DOLocationID']] = df_jan[['PULocationID', 'DOLocationID']].astype(str)
df_jan_ld = df_jan[['PULocationID', 'DOLocationID']].to_dict('records')

In [19]:
v = DictVectorizer()

In [20]:
df_jan_dv = v.fit_transform(df_jan_ld)

In [21]:
df_jan_dv.shape

(2421440, 515)

In [22]:
print(f"Number of columns of the feature matrix = {df_jan_dv.shape[1]}")

Number of columns of the feature matrix = 515


In [23]:
# df_jan_fm = df_jan_dv.todense()

In [24]:
# df_jan_fm.shape

In [25]:
# v.get_feature_names_out()

## Q5

In [26]:
from sklearn.linear_model import LinearRegression

In [27]:
X = df_jan_dv
y = df_jan['duration']

In [28]:
lr = LinearRegression().fit(X, y)

In [29]:
from sklearn.metrics import mean_squared_error

In [30]:
y_pred = lr.predict(X)

In [31]:
rmse = mean_squared_error(y, y_pred, squared=False)

In [32]:
print(f"RMSE on train is {rmse}")

RMSE on train is 6.98619123059128


## Q6

In [33]:
df_feb = pd.read_parquet(path='https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

df_feb = get_duration(df_feb)

df_feb = remove_outliers(df_feb)

In [34]:
df_feb[['PULocationID', 'DOLocationID']] = df_feb[['PULocationID', 'DOLocationID']].astype(str)
df_feb_ld = df_feb[['PULocationID', 'DOLocationID']].to_dict('records')

In [35]:
X_feb = v.transform(df_feb_ld)

In [36]:
y_feb = df_feb['duration']

In [37]:
y_feb_pred = lr.predict(X_feb)

In [38]:
rmse_validation = mean_squared_error(y_feb, y_feb_pred, squared=False)

In [39]:
print(f"RMSE on validation is {rmse_validation}")

RMSE on validation is 7.786413522518241
